In [2]:
from sklearn.neighbors import KDTree
import numpy as np
import networkx as nx
import quaternion as qt
import open3d as o3d
"""
load pose
build kdtree
for each pose find nearest neighbor
get edge based on orientation as well

check connected
save graph
"""

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'\nload pose\nbuild kdtree\nfor each pose find nearest neighbor\nget edge based on orientation as well\n\ncheck connected\nsave graph\n'

In [3]:
pose_path = "/media/hieu/T7/3d-front/poses_100_2/00ad8345-45e0-45b3-867d-4a3c88c2517a.npy"

pose = np.load(pose_path)
pose[:,3,3] = 1
pose.shape

(1763, 4, 4)

In [4]:
def make_edges(poses, distance, angle):
    """
    2 pose is connected if they are within distance and angle
    camera coordinate is x right, y up, z backward
    """
    cosine_thr = np.cos(np.radians(angle))
    # make graph based on distance only
    kdtree = KDTree(poses[:, :3, 3], leaf_size=30, metric='euclidean')
    
    # forward direction of the pose
    ## came forward is -z
    cam_forward = np.array([0, 0, -1])
    world_forward = np.einsum('nij, j->ni', poses[:, :3, :3], cam_forward)
    
    edges = set()
    # select edge based angle
    for i in range(poses.shape[0]-1):
        # find nearest neighbor
        neighbor_indices = kdtree.query_radius(poses[i:i+1, :3, 3], r=distance)[0]
        # only need to check for index > i to reduce redundancy
        neighbor_indices = neighbor_indices[neighbor_indices > i]
        if not len(neighbor_indices):
            continue
        # check angle between forward direction
        neighbor_directions = world_forward[neighbor_indices]
        cosines = np.einsum('ni, i -> n', neighbor_directions, world_forward[i])
        selected_indices = neighbor_indices[cosines >= cosine_thr]
        for j in selected_indices:
            edges.add((i, j))
            edges.add((j, i))
    return edges

def make_graph(poses, edges):
    G = nx.Graph()
    for i, pose in enumerate(poses):
        G.add_node(f"{i:0>4}", pose=pose)
    for i,j in edges:
        G.add_edge(f"{i:0>4}", f"{j:0>4}")
    return G

In [5]:
edges = make_edges(pose, 0.5, 60)
print(len(edges))
G = make_graph(pose, edges)


8098


In [124]:
nx.write_gpickle(G, "graph.pkl")
GG = nx.read_gpickle("graph.pkl")

In [126]:
len(GG.edges), len(G.edges)

(228, 228)

In [127]:
GG.nodes["0000"]['pose']

array([ 1.56168006,  1.36513442, -7.18073192])

In [115]:
(0.2**2+0.5**2)**0.5

0.5385164807134505

In [116]:
# visualize
o3d_pcd = o3d.geometry.PointCloud()
o3d_pcd.points = o3d.utility.Vector3dVector(pose[:, :3, 3])

o3d_lines = o3d.geometry.LineSet()
o3d_lines.points = o3d.utility.Vector3dVector(pose[:, :3, 3])
lines = np.array(list(edges))
o3d_lines.lines = o3d.utility.Vector2iVector(lines)

# make camera
K = np.array([[256.0, 0, 256], [0, 256, 256], [0, 0, 1]], dtype=np.float64)
# K = o3d.camera.PinholeCameraIntrinsic(512, 512, 256, 256, 256, 256)
cameras = []

w2c = np.linalg.inv(pose.astype(np.float64))
for p in w2c:
    camera = o3d.geometry.LineSet.create_camera_visualization(
        view_width_px=512, view_height_px=512, intrinsic=K, extrinsic=p, scale = 0.05
    )
    cameras.append(camera)

o3d.visualization.draw_geometries([o3d_pcd, o3d_lines]+cameras)
# o3d.visualization.draw_geometries(cameras)

In [6]:
# check if point is connected
isolated_nodes = set(list(nx.isolates(G)))
nodes = set(G.nodes) - isolated_nodes
single = [node for node in nodes if G.degree(node) == 1]
nodes = nodes - set(single)
subgraph = G.subgraph(nodes)
nx.is_connected(subgraph)

False

In [7]:
def get_connected_subgraph(G:nx.Graph, n_nodes:int):
    """
    get list of connected subgraphs with given size of a graph
    args:
        G: nx.Graph
        n_nodes: size of subgraph
    return list of connected subgraphs, each subgraph is a list of nodes
    """
    
    def recursive_local_expand(node_set, possible, excluded, results, max_size):
        """
        expand the subgraph until reach the max size
        using excluded to avoid permutation of the same subgraph
        """
        if len(node_set) == max_size:
            results.append(node_set)
            return
        for j in sorted(list(possible - excluded)):
            new_node_set = node_set | {j}
            excluded = excluded | {j}
            new_possible = (possible | set(G.neighbors(j))) - excluded
            recursive_local_expand(new_node_set, new_possible, excluded, results, max_size)
    results = []
    excluded = set()
    nodes = list(G.nodes)
    for node in nodes:
        excluded.add(node)
        recursive_local_expand({node}, set(G.neighbors(node))-excluded, excluded, results, n_nodes)
    return results

In [14]:
subgraph = get_connected_subgraph(G, 6)
len(subgraph)

1049174

In [15]:
cc = [c for c in nx.connected_components(G)]

In [16]:
len(cc)

65

In [17]:
len([c for c in cc if len(c) >= 6])

8

In [19]:
def recursive_expand(
    G,
    node_set,
    possible,
    excluded,
    results,
    max_size,
    deterministic=True,
    return_one=False,
):
    """
    expand the subgraph until reach the max size
    using excluded to avoid permutation of the same subgraph
    """
    if return_one and len(results) > 0:
        return

    if len(node_set) == max_size:
        results.append(node_set)
        return
    candidates = sorted(list(possible - excluded))
    if not deterministic:
        permutation = np.random.permutation(len(candidates))
        candidates = [candidates[i] for i in permutation]
    for j in candidates:
        new_node_set = node_set | {j}
        excluded = excluded | {j}
        new_possible = (possible | set(G.neighbors(j))) - excluded
        recursive_expand(G, new_node_set, new_possible, excluded, results, max_size)

In [93]:
total_frames = 6
cc = [
    c
    for c in nx.connected_components(G)
    if len(c) >= total_frames
]
nodes = sum([list(c) for c in cc], [])
GG = G.subgraph(nodes).copy()

In [148]:
# sample subgraph



start_node = sorted(list(GG.nodes))[1000]
print(start_node)
subgraphs = []
exclude = {start_node}
deterministic = False

recursive_expand(
    GG,
    {start_node},
    set(GG.neighbors(start_node)) - exclude,
    exclude,
    subgraphs,
    total_frames,
    deterministic,
    return_one=True,
)
sorted(subgraphs[0])

1044


['0883', '0884', '0907', '0940', '1002', '1044']

In [119]:
# viz image of edge 

# edges for location, panorama settings

In [117]:
location_path = "/media/hieu/T7/3dfront_render/locations_secondbedroom/6a0e73bc-d0c4-4a38-bfb6-e083ce05ebe9.npy"
location_path = "/media/hieu/T7/3d-front/locations_100_7/00ad8345-45e0-45b3-867d-4a3c88c2517a.npy"
locations = np.load(location_path)
locations.shape

(132, 3)

In [123]:
132*12

1584

In [118]:
(0.7**2+0.5**2)**0.5

0.8602325267042626

In [119]:
def make_edges_loc(locations, distance):
    """
    2 location is connected if they are within distance
    """
    # make graph based on distance only
    kdtree = KDTree(locations, leaf_size=30, metric='euclidean')
    
    # forward direction of the pose
    ## came forward is -z
    # cam_forward = np.array([0, 0, -1])
    # world_forward = np.einsum('nij, j->ni', poses[:, :3, :3], cam_forward)
    
    edges = set()
    # select edge based angle
    for i in range(locations.shape[0]-1):
        # find nearest neighbor
        neighbor_indices = kdtree.query_radius(locations[i:i+1], r=distance)[0]
        # only need to check for index > i to reduce redundancy
        neighbor_indices = neighbor_indices[neighbor_indices > i]
        if not len(neighbor_indices):
            continue
        for j in neighbor_indices:
            edges.add((i, j))
            edges.add((j, i))
    return edges

In [120]:
edges = make_edges_loc(locations, 1.0)
print(len(edges))
G = make_graph(locations, edges)

456


In [121]:
def get_camera_poses(location: np.ndarray) -> np.ndarray:
    """
    select poses from the given location

    try 2 settings
        12 views, 0 deg elevation
        12 views, 6 look down 30 deg 6 look up 30 deg

    NOTE: assume y is up
    # cam coor in blender is x right, y up, z backward
    """
    HEADINGS = np.arange(12) * 2 * np.pi / 12
    # rotation for setting 1
    ROTATIONS1 = qt.from_rotation_vector(HEADINGS[:, np.newaxis] * np.array([0, 1, 0]))
    ROTATIONS1 = qt.as_rotation_matrix(ROTATIONS1)

    poses = np.zeros((len(HEADINGS), 4, 4), dtype=np.float32)
    poses[:, :3, :3] = ROTATIONS1
    poses[:, :3, 3] += location
    poses[:, 3, 3] = 1
    return poses

In [122]:
# visualize
o3d_pcd = o3d.geometry.PointCloud()
o3d_pcd.points = o3d.utility.Vector3dVector(locations)

o3d_lines = o3d.geometry.LineSet()
o3d_lines.points = o3d.utility.Vector3dVector(locations)
lines = np.array(list(edges))
o3d_lines.lines = o3d.utility.Vector2iVector(lines)

# make camera
K = np.array([[256.0, 0, 256], [0, 256, 256], [0, 0, 1]], dtype=np.float64)
cameras = []

for location in locations:
    poses = get_camera_poses(location)
    w2c = np.linalg.inv(poses.astype(np.float64))
    for p in w2c:
        camera = o3d.geometry.LineSet.create_camera_visualization(
            view_width_px=512, view_height_px=512, intrinsic=K, extrinsic=p, scale = 0.05
        )
        cameras.append(camera)

o3d.visualization.draw_geometries([o3d_pcd, o3d_lines]+cameras)
# o3d.visualization.draw_geometries([o3d_pcd, o3d_lines])